### ![](../../img/installation-ico.png) Installation

In [ ]:
pip install langchain langchain-community langchain-google-vertexai google.auth

### ![](../../img/package-ico.png) Imports

In [ ]:
import os
import google.auth

from langchain_google_vertexai import VertexAI

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.base import BaseCallbackHandler

### ![](../../img/parametrage-ico.png) Paramétrages

In [ ]:
# Gestion credentials VertexAI

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../settings/credentials.json"
google.auth.default()

#### Callbacks

In [ ]:
from typing import Union, Any, Dict
models = [ "gemini-1.5-pro", "text-bison", "text-unicorn" ]

class MyHandler(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts, metadata, **kwargs
    ):
        print(f"on_llm_start {metadata['name']}")
        
    def on_llm_end(
        self, response, **kwargs
    ):
        print(f"on_llm_end")

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when LLM errors."""
        print("Got error while running :", error)

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], metadata, **kwargs: Any
    ) -> Any:
        print(f"on_chain_start {metadata['name']}")
        
    def on_chain_end(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], metadata, **kwargs: Any
    ) -> Any:
        print(f"on_chain_end {metadata['name']}")

llm = VertexAI(
    model_name="text-bison",
    temperature=0.1
)

llms = [ VertexAI(model_name=m, callbacks=[MyHandler()], metadata={"name": m}) for m in models]
prompt = PromptTemplate.from_template(
    "Tell me a joke about {input}."
)
output_parser = StrOutputParser()

chains = [prompt | llm | output_parser for llm in llms]

### ![](../../img/jouer-ico.png) Exécution(s)

In [ ]:
input = "elephant"
for c in chains:
    for s in c.invoke({"input": input}):
        print(s, end="", flush=True)

#### Configurable alternatives

In [ ]:
from langchain_google_vertexai import ChatVertexAI, VertexAI
from langchain_core.runnables import ConfigurableField
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate

llm = ChatVertexAI(
    model_name="gemini-1.5-pro",
    temperature=0
).configurable_alternatives(
    ConfigurableField(id="llm"),
    default_key="latest",
    # Ajout des autres options associées à d'autre clé
    gemini_old=ChatVertexAI(model_name="gemini-1.0-pro"),
    legacy=VertexAI(model_name="text-bison")
)

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""En tant qu'assistant IA, avant toute réponse indique absolument qui tu es et le nom exact de ton modèle.
                  Uniquement et seulement après, tu peux répondre à la demande de l'utilisateur.
                  Ta réponse sera sous la forme suivante :
                  Modèle : 'Nom du modèle utilisé'
                  Réponse : 'Réponse à la demande'
                  """),
    HumanMessage(content="Parle moi de LangChain.")
    ])
chain = prompt | llm

configs = [{"llm": model_key} for model_key in ["latest", "gemini_old", "legacy"]]
for c in configs:
    print("Calling with ", c)
    print(chain.with_config(configurable=c).invoke({}))
    print("------")
